# A/B testing: conversion rate

Loading data from disk

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("data/raw_data.csv")
print("Number of visits =", df.shape[0])
df.head()

Number of visits = 294478


,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


## Exploratory Data Analysis and Data preparation

Let's primarally explore our features. For each feature we want to display:
 - number of unique values, 
 
 - range of values, 
 
 - number of missing values, 
 
 - a few examples of values

In [2]:
summary = pd.DataFrame(index=df.columns, columns=["# unique", "range", "# missing", "examples"])
for name in df.columns:
    summary.loc[name, "# unique"] = df[name].nunique()
    summary.loc[name, "range"] = [df[name].max(), df[name].min()]
    summary.loc[name, "# missing"] = df[name].isna().sum()
    summary.loc[name, "examples"] = df[name].unique()[:2]
summary

,# unique,range,# missing,examples
user_id,290584,"[945999, 630000]",0,"[851104, 804228]"
timestamp,294478,"[2017-01-24 13:41:54.460509, 2017-01-02 13:42:...",0,"[2017-01-21 22:11:48.556739, 2017-01-12 08:01:..."
group,2,"[treatment, control]",0,"[control, treatment]"
landing_page,2,"[old_page, new_page]",0,"[old_page, new_page]"
converted,2,"[1, 0]",0,"[0, 1]"


Next, we should figure out, how features "group" and "landing_page" are connected. 
We expect each user from control group to land on the old page. Analogically for users from treatment group.

In [3]:
pd.crosstab(df['group'], df['landing_page'])

landing_page,new_page,old_page
group,,
control,1928,145274
treatment,145311,1965


Ooops. Apparently, there are users that landed at wrong page by mistake. Removing visits by mistake:

In [4]:
mask =  ((df["group"] == "control")   & (df["landing_page"] =="old_page")) | \
        ((df["group"] == "treatment") & (df["landing_page"] =="new_page"))

df = df[mask]

# check: must not contain "new_page"
print(df[df["group"] == "control"]["landing_page"].unique())

['old_page']


Okay. 

We go further. There may be users with 2 or more visits.

In [5]:
counts = pd.DataFrame(df["user_id"], index = df.index)
counts = counts["user_id"].value_counts()
ids = counts[counts > 1].index.values
df_recurring = df.loc[df['user_id'].isin(ids)]

counts.head()

773192    2
630732    1
811737    1
797392    1
795345    1
Name: user_id, dtype: int64

Strategy: delete all duplicate visits except one.

In [6]:
ind_to_drop = []
for user_id in ids:
    for group in ["control", "treatment"]:
        for landing_page in ["old_page", "new_page"]:
            for converted in [0, 1]:
                arr  = (df_recurring.loc[(df_recurring["user_id"]==user_id) & 
                                        (df_recurring["group"]==group) & 
                                        (df_recurring["landing_page"]==landing_page) & 
                                        (df_recurring["converted"]==converted)]
                                    .index[:1]
                       )
                if len(arr) == 1:
                    ind_to_drop.append(arr[0])
                    
print("We drop the following recurring visits : ", ind_to_drop)
df = df[~df["user_id"].isin(ind_to_drop)]

We drop the following recurring visits :  [1899]


**Next,we want to detect seasonality in time series of CR.**

In [7]:
df["day"] = pd.to_datetime(df["timestamp"]).dt.day

conv_days = (df[["day", "group", "converted"]]
              .groupby(["group", "day"])
              .agg(["mean", "size"])
             )
conv_days.columns = ["conv_rate", "size"]
control_days = conv_days.loc["control"]
treatment_days = conv_days.loc["treatment"]
conv_days.head()

conv_rate  size
group   day                 
control 2     0.125568  2859
        3     0.113809  6590
        4     0.121922  6578
        5     0.123230  6427
        6     0.115350  6606


The plot was generated by [Plots notebook](https://github.com/pbilinskiy/AB-testing/blob/master/Plots.ipynb). 
Red regions - Saturday and Sunday. We expect significant changes on these days.

<img src="plots/CR_time_series.png">



There may be increasing of CR of control group and decresing CR of treatment group in 14 and 15 January (Saturday and Sunday).

Let's figure it out. Grouping data by days of week:

In [8]:
df["day"] = pd.to_datetime(df["timestamp"]).dt.dayofweek

conv_days = (df[["day", "group", "converted"]]
              .groupby(["group", "day"])
              .agg(["mean", "size"])
             )

conv_days.columns = ["conv_rate", "size"]
conv_days.reset_index(inplace=True)
conv_days


,group,day,conv_rate,size
0,control,0,0.122795,22794
1,control,1,0.116748,23615
2,control,2,0.121835,19748
3,control,3,0.121729,19527
4,control,4,0.115834,19692
5,control,5,0.124567,19901
6,control,6,0.119518,19997
7,treatment,0,0.119491,22646
8,treatment,1,0.122339,23533
9,treatment,2,0.118837,19817


The plots were generated by [Plots notebook](https://github.com/pbilinskiy/AB-testing/blob/master/Plots.ipynb). 

<img src="plots/CR_daysofweek_A.png">
<img src="plots/CR_daysofweek_B.png">



**We don't observe any seasonality by week day.**

Saving the cleaned data.

In [9]:
df.to_csv("cleaned_data.csv")

## Visualization: conversion rate of two groups


The plot was generated by [Plots notebook](https://github.com/pbilinskiy/AB-testing/blob/master/Plots.ipynb). 

<img src = "plots/conversion_rates.png">

## Testing the hypothesis

Before performing statistical test, we should calculate a required size of sample. 

Our goal is to be able to detect difference in 0.02 with probability 0.8 (it's called a power of test). Another parameter we mush choose by hands is $\alpha$ - the level of significance (probability of type I error). Common choise is $\alpha = 0.05$.

In [10]:
# compute standartized effect size
from statsmodels.stats.proportion import proportion_effectsize

agg = df[["converted", "group"]].groupby("group").aggregate(["mean", "size", "sum"])
agg.columns = ["conv_rate", "size", "count"]

n1 = agg.loc["control", "size"]
n2 = agg.loc["treatment", "size"]
p1 = agg.loc["control", "conv_rate"]
p2 = agg.loc["treatment", "conv_rate"]
# we want to capture change  in 0.02
effect_size = proportion_effectsize(p1, p1+0.02)

# calculating required sqmple size
from statsmodels.stats.power import NormalIndPower
alpha = 0.05
print("alpha      = {:.3f}".format(alpha))
# assume standard deviations of p1 and p2 are equal 
required_n = NormalIndPower().solve_power(effect_size=effect_size, 
                                          alpha=alpha, 
                                          nobs1=None, 
                                          ratio=1, 
                                          power=0.8)
required_n = np.ceil(required_n).astype(int)
print("required n =", required_n)

control_sample = df[df["group"]=="control"].sample(required_n)
treatment_sample = df[df["group"]=="treatment"].sample(required_n)

alpha      = 0.050
required n = 4444


Next - performing z-test. Our null hypothesis:
$$H_0: p_1 = p_2$$
with two-tailed alternative:
$$H_1: p_1 \neq p_2$$

In [11]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint


count = [control_sample["converted"].sum(),
         treatment_sample["converted"].sum()]
nobs = [control_sample.shape[0],
        treatment_sample.shape[0]]
# perform z-test
z_stat, p_val = proportions_ztest(count, nobs)
# confidence intervals for true proportions
(lower_con, lower_treat), (upper_con, upper_treat) = proportion_confint(count, nobs, alpha)

Reporting the results:

In [12]:
print("z-statistic  = {:.3f}".format(z_stat))
print("p-value      = {:.3f}".format(p_val))
print("{:}% ci for conversion rate of control group   = ({:.4f}, {:.4f})"
      .format(int((1-alpha)*100), lower_con, upper_con))
print("{:}% ci for conversion rate of treatment group = ({:.4f}, {:.4f})"
      .format(int((1-alpha)*100), lower_treat, upper_treat))


z-statistic  = 0.428
p-value      = 0.668
95% ci for conversion rate of control group   = (0.1093, 0.1283)
95% ci for conversion rate of treatment group = (0.1065, 0.1253)


Now we can plot CR's with confidence intervels. As you can see, no significant difference is present.

<img src = "plots/conversion_rates_CI.png">

The plot was generated by [Plots notebook](https://github.com/pbilinskiy/AB-testing/blob/master/Plots.ipynb). 

## Conclusion

p-value is much larger than $\alpha=0.05$.
Therefore, we can not reject the null hypothesis.

We conclude that:
1. **Conversions rates in both groups ARE THE SAME**

2. **Data doesn't show significant difference between two groups.**